# A Bayesian Approach to the Titanic Data Set

A recent obsession of mine has been Bayesian Neural Networks.  We will apply this approach in two ways. First in a pure NN solution which utilizes dropout.  Second, we'll use edward to express our uncertainty over the weights and sample from the posterior.

In [3]:
# core python
from itertools import product
import re

# Data Structures
import pandas as pd
import numpy as np

# Data Visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Prediction
import tensorflow as tf
import edward as ed
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from edward.models import Normal
np.random.seed(606)

The following functions encompass a data cleaning pipeline. The function preproc at the end wraps the rest so that a single function call will return the desired data set. 

In [4]:
# Imports data and splits Data into training and test sets
def split_and_clean():
    X, y = select_features(pd.read_csv('train.csv'))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 606, stratify = y)
    return X_train, y_train, X_test, y_test

# Select the features of interest. 
def select_features(data):
    target = ['Survived']
    features = ['Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
    dropped_features = ['Cabin', 'Ticket']
    X = data[features].drop(dropped_features, axis=1)
    y = data[target]
    return X, y

# Fill na's with the mean (in the case of fare), and with C in the case of embarked.
def fix_na(data):
    na_vars = {"Fare" : data.Fare.mean(), "Embarked" : "C"}
    return data.fillna(na_vars)

# Processes categorical data into dummy vars
def create_dummies(data, cat_vars, cat_types):
    cat_data = data[cat_vars].values
    for i in range(len(cat_vars)):   
        bins = LabelBinarizer().fit_transform(cat_data[:, 0].astype(cat_types[i]))
        cat_data = np.delete(cat_data, 0, axis=1)
        cat_data = np.column_stack((cat_data, bins))
    return cat_data

# Processes numeric data 
def standardize(data, real_vars):
    real_data = data[real_vars]
    scale = StandardScaler()
    return scale.fit_transform(real_data)

# Extract titles from the Name field and create appropriate One Hot Encoded Columns
def extract_titles(data):
    title_array = data.Name
    first_names = title_array.str.rsplit(', ', expand=True, n=1)
    titles = first_names[1].str.rsplit('.', expand=True, n=1)
    known_titles = ['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer']
    for title in known_titles:
        try:
            titles[title] = titles[0].str.contains(title).astype('int')
        except:
            titles[title] = 0
    return titles.drop([0,1], axis=1).values

# Multilayer Perceptron for filling in ages
def age_model(features=27, n_layers=15, n_hidden = 256, dropout = 0.25, optimizer=Adam()):
    model = Sequential()
    model.add(Dense(n_hidden, input_shape=(features, ), activation='relu', kernel_initializer='random_normal'))
    model.add(Dropout(dropout))    
    for i in range(n_layers):
        model.add(Dense(n_hidden, activation='relu'))
        model.add(Dropout(dropout))           
    model.add(Dense(1))   
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])
    return model

# Train the age model and fill in those missing values in the dataset
def impute_ages(data):
    known = data[np.isnan(data[:, -1].astype('float'))==False]
    unknown = data[np.isnan(data[:, -1].astype('float'))]
    y = known[:, -1]
    X = known[:, :-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 606)
    model = age_model()
    model.fit(X_train, y_train, batch_size=64, epochs = 50,
              verbose = 0, validation_split = 0.2)
    ages_predicted = model.predict(unknown[:, :-1])
    data[np.isnan(data[:, -1].astype('float'))] = ages_predicted
    return data

# Executes the full preprocessing pipeline.
def preproc():
    # Import Data & Split
    X_train_, y_train, X_test_, y_test = split_and_clean()
    # Fill NAs
    X_train, X_test = fix_na(X_train_), fix_na(X_test_)
    # Preproc Categorical Vars
    cat_vars = ['Pclass', 'Sex', 'Embarked']
    cat_types = ['int', 'str', 'str']
    X_train_cat, X_test_cat = create_dummies(X_train, cat_vars, cat_types), create_dummies(X_test, cat_vars, cat_types)
    # Preprocess Numeric Vars
    real_vars = ['Fare', 'SibSp', 'Parch']
    X_train_real, X_test_real = standardize(X_train, real_vars), standardize(X_test, real_vars)
    # Extract Titles
    X_train_titles, X_test_titles = extract_titles(X_train), extract_titles(X_test)
    # Recombine
    X_train, X_test = np.column_stack((X_train_cat, X_train_real, X_train_titles, X_train_.Age)), np.column_stack((X_test_cat, X_test_real, X_test_titles, X_test.Age))
    # Fill Missing Ages
    X_train, X_test = impute_ages(X_train), impute_ages(X_test)
    return X_train, np_utils.to_categorical(y_train.values), X_test, np_utils.to_categorical(y_test.values)

Run the preproc pipeline

In [5]:
X_train, y_train, X_test, y_test = preproc()

Now we can build a Keras model.  At the top we define a series of variables that we'll use in the model.

In [63]:
def create_model(features=28, n_layers=10, n_hidden = 64, dropout = 0.3, optimizer=Adam()):
    model = Sequential()
    model.add(Dense(n_hidden, input_shape=(features, ), activation='relu', kernel_initializer='random_normal'))
    model.add(Dropout(dropout))    
    for i in range(n_layers):
        model.add(Dense(n_hidden, activation='relu'))
        model.add(Dropout(dropout))           
    model.add(Dense(2, activation='softmax'))   
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
    return model

### The last thing to do is to compile and fit the model.

In [6]:
def fit_model(n_layers=14, n_hidden=64, dropout=0.3, epochs=200):
    model = create_model(features=X_train.shape[1], n_layers=n_layers, n_hidden=n_hidden, dropout=dropout)
    model.fit(X_train, y_train , epochs=epochs, batch_size = 64)
    return model

param_grid = {
    'n_layers' : [5, 10],
    'n_hidden' : [25, 75],
    'dropout'  : [0.25, 0.35]
}

In [7]:
model = KerasClassifier(build_fn=create_model, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)

Epoch 1/10
445/445 [==============================] - 0s - loss: 0.6903 - acc: 0.5865      
Epoch 2/10
445/445 [==============================] - 0s - loss: 0.6734 - acc: 0.6292     
Epoch 3/10
445/445 [==============================] - 0s - loss: 0.6555 - acc: 0.6315     
Epoch 4/10
445/445 [==============================] - 0s - loss: 0.6310 - acc: 0.6315     
Epoch 5/10
445/445 [==============================] - 0s - loss: 0.6073 - acc: 0.6315     
Epoch 6/10
445/445 [==============================] - 0s - loss: 0.5923 - acc: 0.6315     
Epoch 7/10
445/445 [==============================] - 0s - loss: 0.5906 - acc: 0.6315     
Epoch 8/10
445/445 [==============================] - 0s - loss: 0.5668 - acc: 0.6315     
Epoch 9/10
445/445 [==============================] - 0s - loss: 0.5662 - acc: 0.6315     
Epoch 10/10
445/445 [==============================] - 0s - loss: 0.6892 - acc: 0.5843     
Epoch 2/10
445/445 [==============================] - 0s - loss: 0.6779 - acc: 0.5820   

446/446 [==============================] - 0s - loss: 0.3892 - acc: 0.8498     
Epoch 7/10
446/446 [==============================] - 0s - loss: 0.4094 - acc: 0.8520     
Epoch 8/10
446/446 [==============================] - 0s - loss: 0.3753 - acc: 0.8475     
Epoch 9/10
446/446 [==============================] - 0s - loss: 0.3834 - acc: 0.8408     
Epoch 10/10
445/445 [==============================] - 1s - loss: 0.6688 - acc: 0.6202     
Epoch 2/10
445/445 [==============================] - 0s - loss: 0.6120 - acc: 0.6584     
Epoch 3/10
445/445 [==============================] - 0s - loss: 0.5453 - acc: 0.7528     
Epoch 4/10
445/445 [==============================] - 0s - loss: 0.5400 - acc: 0.8045     
Epoch 5/10
445/445 [==============================] - 0s - loss: 0.4879 - acc: 0.8022     
Epoch 6/10
445/445 [==============================] - 0s - loss: 0.4910 - acc: 0.8022     
Epoch 7/10
445/445 [==============================] - 0s - loss: 0.5002 - acc: 0.8180     
Epoch 8/1

446/446 [==============================] - 2s - loss: 0.6798 - acc: 0.6031      
Epoch 2/10
446/446 [==============================] - 0s - loss: 0.6520 - acc: 0.6300     
Epoch 3/10
446/446 [==============================] - 0s - loss: 0.6309 - acc: 0.6278     
Epoch 4/10
446/446 [==============================] - 0s - loss: 0.5925 - acc: 0.6323     
Epoch 5/10
446/446 [==============================] - 0s - loss: 0.5566 - acc: 0.6480     
Epoch 6/10
446/446 [==============================] - 0s - loss: 0.5555 - acc: 0.7466     
Epoch 7/10
446/446 [==============================] - 0s - loss: 0.5352 - acc: 0.7377     
Epoch 8/10
446/446 [==============================] - 0s - loss: 0.5234 - acc: 0.7332     
Epoch 9/10
446/446 [==============================] - 0s - loss: 0.5171 - acc: 0.7803     
Epoch 10/10
445/445 [==============================] - 1s - loss: 0.6671 - acc: 0.6090     
Epoch 2/10
445/445 [==============================] - 0s - loss: 0.6018 - acc: 0.6247     
Epoch 3/

In [ ]:
best_model = grid_result.best_estimator_.model
train_score, train_accuracy = best_model.evaluate(X_train, y_train)
test_score, test_accuracy = best_model.evaluate(X_test, y_test)
print('Training Score: {0}, Trainng Accuracy: {1}'.format(train_score, train_accuracy))
print('Test Score: {0}, Test Accuracy: {1}'.format(test_score, test_accuracy))

In [ ]:
print(grid_result.best_params_)
plt.hist(grid_result.cv_results_['mean_train_score'])

In [17]:
age_data = pd.read_csv('train.csv')
age_unknown = age_data[age_data.isnull()]
age_known = age_data[age_data.isnull() == False]


0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
6      54.0
7       2.0
8      27.0
9      14.0
10      4.0
11     58.0
12     20.0
13     39.0
14     14.0
15     55.0
16      2.0
18     31.0
20     35.0
21     34.0
22     15.0
23     28.0
24      8.0
25     38.0
27     19.0
30     40.0
33     66.0
34     28.0
35     42.0
37     21.0
38     18.0
       ... 
856    45.0
857    51.0
858    24.0
860    41.0
861    21.0
862    48.0
864    24.0
865    42.0
866    27.0
867    31.0
869     4.0
870    26.0
871    47.0
872    33.0
873    47.0
874    28.0
875    15.0
876    20.0
877    19.0
879    56.0
880    25.0
881    33.0
882    22.0
883    28.0
884    25.0
885    39.0
886    27.0
887    19.0
889    26.0
890    32.0
Name: Age, Length: 714, dtype: float64

In [ ]:
def preproc_testing():
    X = pd.read_csv('test.csv')
    # Fill NAs
    X = fix_na(X)
    # Preproc Categorical Vars
    cat_vars = ['Pclass', 'Sex', 'Embarked']
    cat_types = ['int', 'str', 'str']
    X_cat = create_dummies(X, cat_vars, cat_types)
    # Preprocess Numeric Vars
    real_vars = ['Age', 'Fare', 'SibSp', 'Parch']
    X_real = standardize(X, real_vars) 
    # Extract Titles
    X_titles = extract_titles(X)
    # Recombine
    X = np.column_stack((X_cat, X_real, X_titles))
    return X

In [ ]:
testing = preproc_testing()
prediction = grid_result.predict(testing)

In [ ]:
submission = pd.DataFrame()
submission['PassengerId'] = pd.read_csv('test.csv').PassengerId
submission['Survived'] = prediction

In [ ]:
submission.to_csv('keras_titanic.csv', index=False)